In [1]:
import tensorflow as tf

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
df_cxr_data = pd.read_csv('e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full/Data_Entry_2017.csv')
df_cxr_data

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


In [4]:
data_path = 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full/'
all_xray_df = pd.read_csv(data_path+'Data_Entry_2017.csv')

In [6]:
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join(data_path, 'images*', '*', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])

Scans found: 112120 , Total Headers 112120


In [7]:
all_image_paths

{'00000032_007.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_007.png',
 '00000032_008.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_008.png',
 '00000032_009.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_009.png',
 '00000032_010.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_010.png',
 '00000032_011.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_011.png',
 '00000032_012.png': 'e:Anil/edu/DataScience/MTech/22_IndividualProject/05_X-Ray_Report_Generation/Data/NIH/NLMCXR_png_full\\images_001\\images\\00000032_012.png',
 '00000032_013.p

In [16]:
print(all_image_paths.values())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['Patient Age'] = all_xray_df['Patient Age'].map(lambda x: int(x[:-1]))
all_xray_df.sample(3)

TypeError: 'int' object is not subscriptable

In [12]:
df_cxr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112120 entries, 0 to 112119
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Image Index                  112120 non-null  object 
 1   Finding Labels               112120 non-null  object 
 2   Follow-up #                  112120 non-null  int64  
 3   Patient ID                   112120 non-null  int64  
 4   Patient Age                  112120 non-null  int64  
 5   Patient Gender               112120 non-null  object 
 6   View Position                112120 non-null  object 
 7   OriginalImage[Width          112120 non-null  int64  
 8   Height]                      112120 non-null  int64  
 9   OriginalImagePixelSpacing[x  112120 non-null  float64
 10  y]                           112120 non-null  float64
 11  Unnamed: 11                  0 non-null       float64
dtypes: float64(3), int64(5), object(4)
memory usage: 10.3+ MB


In [13]:
df_cxr_data.drop(['Unnamed: 11'], axis=1, inplace=True)
df_cxr_data

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168
